# Module 9. Scalable Data Processing

## Advanced Task 2: Analysis of access to electricity.

### Pablo Yañez Martin

I will do this task using RDDs.

In [1]:
from pyspark import SparkConf, SparkContext
import numpy as np

In [2]:
ruta1 = "D:/Dropbox/Pablo/Master/Modulo 9. Scalable Data Processing/Java/Data/Electricidad.csv"

In [3]:
ruta2 = "D:/Dropbox/Pablo/Master/Modulo 9. Scalable Data Processing/Java/Data/Metadata_Country.csv"

In [4]:
spark_conf = SparkConf().setAppName("Task_Advanced_ElectricityRDD").setMaster("local[2]")
spark_context = SparkContext(conf=spark_conf)

In [5]:
logger = spark_context._jvm.org.apache.log4j
logger.LogManager.getLogger("org").setLevel(logger.Level.WARN)

In [6]:
lectura1 = spark_context.textFile(ruta1)

I will delete the first lines, because the data starts in the fifth line. 

In [7]:
line1 = lectura1.first()
lectura1 = lectura1.filter(lambda line : line!=line1)

line2 = lectura1.first()
lectura1 = lectura1.filter(lambda line : line!=line2)
    
line3 = lectura1.first()
lectura1 = lectura1.filter(lambda line : line!=line3)
    
line4 = lectura1.first()
lectura1 = lectura1.filter(lambda line : line!=line4)

The range of years will be the following:

In [8]:
init = 2000
fin = 2010

The n-th column where starts the range of years is:

In [9]:
n = init - 1960 + 4

In [10]:
columnas1=lectura1.map(lambda array : array.split(';'))

I will do a firts filter of the data and keep with the data of the range of years that I have selected before.

In [11]:
data1 = columnas1\
        .filter(lambda array : len(array[1])==3)\
        .map(lambda array : [array[1],array[n],array[n+1],array[n+2],array[n+3],array[n+4],array[n+5],array[n+6],array[n+7],array[n+8],array[n+9],array[n+10]])\
        .collect()

With the following function, I will edit the null values and the values that are written wrongly, such as '937.373.377.234'.

In [12]:
def toFloat(x):
        try:
            float(x)
            return float(x)
        except ValueError:
            if (x==''):
                return 0.0
            elif(x[1]!='.' and x[2]!='.'):
                return float(x[0]+x[1]+'.'+x[2])
            else:
                return float(x[0]+x[1]+x[2])


In order to make this changes I have apply the function collect to the rdd, and when I have finished that, I will take again a rdd and keep with the average of electricity in the range of years chosen.

In [13]:
for x in data1:
    for i in range(1,12):
        x[i]=toFloat(x[i])

In [14]:
data1=spark_context.parallelize(data1)\
        .map(lambda array : [array[0],np.mean([array[1],array[2],array[3],array[4],array[5],array[6],array[7],array[8],array[9],array[10],array[11]])])


In [15]:
for x in data1.collect():
    print(x)

['ABW', 98.62727272727274]
['AFG', 19.654545454545453]
['AGO', 26.354545454545452]
['ALB', 0.0]
['AND', 100.0]
['ARB', 75.21818181818182]
['ARE', 100.0]
['ARG', 80.24727272727273]
['ARM', 99.28181818181817]
['ASM', 0.0]
['ATG', 53.5]
['AUS', 100.0]
['AUT', 100.0]
['AZE', 99.37272727272726]
['BDI', 34.300000000000004]
['BEL', 100.0]
['BEN', 26.799999999999997]
['BFA', 26.61818181818182]
['BGD', 44.50818181818182]
['BGR', 100.0]
['BHR', 100.0]
['BIH', 99.71818181818183]
['BLR', 100.0]
['BLZ', 84.9909090909091]
['BMU', 100.0]
['BOL', 75.58181818181819]
['BRA', 96.84545454545454]
['BRB', 99.98181818181818]
['BRN', 100.0]
['BTN', 55.58636363636363]
['BWA', 37.3909090909091]
['CAF', 53.054545454545455]
['CAN', 100.0]
['CEB', 100.0]
['CHE', 100.0]
['CHI', 0.0]
['CHL', 98.07272727272726]
['CHN', 98.17272727272729]
['CIV', 53.35454545454545]
['CMR', 47.554545454545455]
['COL', 87.69090909090909]
['COM', 31.65454545454546]
['CPV', 56.51818181818182]
['CRI', 90.49090909090908]
['CSS', 90.35454545

Now, we have the csv API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.csv cleaned and ready to use in the following cells.

In [16]:
lectura2 = spark_context.textFile(ruta2)

In [17]:
header=lectura2.first()
lectura2=lectura2.filter(lambda line : line!=header)

In [18]:
columnas2=lectura2.map(lambda line: line.split(','))

I have used the map function to delete the char " at the end and at the final. I have done this because when I will join the following rdd with the previous rdd, I have to take care with how is each keys, althought has the same information, are written in differents ways, such as, Spain in the firts rdd is 'ESP' and in the following rdd is '"ESP"'.

In [19]:
data2=columnas2\
        .filter(lambda par : len(par[0])==5)\
        .filter(lambda par : len(par[1])!=2)\
        .map(lambda array : [array[0][1]+array[0][2]+array[0][3],array[1]])
       

In [20]:
for x in data2.collect():
    print('Code : '+x[0]+', '+'Region: '+x[1])


Code : ABW, Region: "Latin America & Caribbean"
Code : AFG, Region: "South Asia"
Code : AGO, Region: "Sub-Saharan Africa"
Code : ALB, Region: "Europe & Central Asia"
Code : AND, Region: "Europe & Central Asia"
Code : ARE, Region: "Middle East & North Africa"
Code : ARG, Region: "Latin America & Caribbean"
Code : ARM, Region: "Europe & Central Asia"
Code : ASM, Region: "East Asia & Pacific"
Code : ATG, Region: "Latin America & Caribbean"
Code : AUS, Region: "East Asia & Pacific"
Code : AUT, Region: "Europe & Central Asia"
Code : AZE, Region: "Europe & Central Asia"
Code : BDI, Region: "Sub-Saharan Africa"
Code : BEL, Region: "Europe & Central Asia"
Code : BEN, Region: "Sub-Saharan Africa"
Code : BFA, Region: "Sub-Saharan Africa"
Code : BGD, Region: "South Asia"
Code : BGR, Region: "Europe & Central Asia"
Code : BHR, Region: "Middle East & North Africa"
Code : BHS, Region: "Latin America & Caribbean"
Code : BIH, Region: "Europe & Central Asia"
Code : BLR, Region: "Europe & Central Asia"


In [21]:
union = data1.join(data2)

In [22]:
for (x,y) in union.collect():
    print(y)

(19.654545454545453, '"South Asia"')
(100.0, '"Middle East & North Africa"')
(80.24727272727273, '"Latin America & Caribbean"')
(100.0, '"Europe & Central Asia"')
(100.0, '"North America"')
(0.0, '"Europe & Central Asia"')
(47.554545454545455, '"Sub-Saharan Africa"')
(100.0, '"Europe & Central Asia"')
(100.0, '"Europe & Central Asia"')
(87.69999999999999, '"Latin America & Caribbean"')
(17.954545454545453, '"Middle East & North Africa"')
(100.0, '"Europe & Central Asia"')
(100.0, '"Europe & Central Asia"')
(79.14545454545454, '"Latin America & Caribbean"')
(89.53272727272727, '"East Asia & Pacific"')
(100.0, '"Europe & Central Asia"')
(100.0, '"Europe & Central Asia"')
(44.51818181818182, '"Middle East & North Africa"')
(98.94545454545455, '"Middle East & North Africa"')
(91.16363636363637, '"Europe & Central Asia"')
(99.45454545454545, '"Europe & Central Asia"')
(87.64818181818181, '"Latin America & Caribbean"')
(63.46363636363637, '"Middle East & North Africa"')
(6.281818181818181, '

In [23]:
num_paises_por_region=union\
    .map(lambda par : (par[1][1],1))\
    .reduceByKey(lambda a,b:a+b)

In [24]:
for x in num_paises_por_region.collect():
    print(x)

('"South Asia"', 8)
('"Middle East & North Africa"', 18)
('"Latin America & Caribbean"', 40)
('"Europe & Central Asia"', 58)
('"East Asia & Pacific"', 32)
('"North America"', 3)
('"Sub-Saharan Africa"', 45)


In [25]:
resultado = union\
        .map(lambda par : (par[1][1],par[1][0]))\
        .reduceByKey(lambda a, b : a+b )

In [26]:
for x in resultado.collect():
    print(x[0]+'\t'+'\t'+str(x[1]))

"South Asia"		465.28545454545457
"Middle East & North Africa"		1515.6090909090908
"Latin America & Caribbean"		3374.494545454545
"Europe & Central Asia"		5195.263636363636
"East Asia & Pacific"		2219.721818181818
"North America"		300.0
"Sub-Saharan Africa"		1615.1854545454546


In [27]:
resultado_final=resultado.join(num_paises_por_region)\
    .map(lambda array : (array[0],array[1][0]/array[1][1]))\
    .filter(lambda par : par[1]>=99.0)\
    .sortBy(lambda par : par[1],ascending=False)

In [28]:
i=0
for x in resultado_final.collect():
    if (i==0):
        print('\n')
        print('Years: '+str(init)+'-'+str(fin))
        print('\n')
        print('Region'+'\t'+'\t'+'\t'+'Percentage')
        print('---------------'+'\t'+'\t'+'---------------')
    i+=1
    print(x[0]+'\t'+'\t'+str(x[1])+'%')



Years: 2000-2010


Region			Percentage
---------------		---------------
"North America"		100.0%


In [29]:
spark_context.stop()